<a href="https://colab.research.google.com/github/mdallas1/shared_code/blob/main/L5_3_accuracy_condition_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install octave

# How accurate is our solution (Section 5.5)?

Computing the solution of $A\mathbf{x}=\mathbf{b}$ motivated our study of the LU method. We found that a standard method that works well for small problems, Cramer's method, is impractically inefficient for even modestly sized problems since the cost grows like $\mathcal{O}((n+1)!)$ for an $n\times n$ problem. The LU method provided a much more efficient technique, the cost growing like $\mathcal{O}(n^3)$, *much* slower than Cramer's rule.

Note that we're only considering what method we use to solve this problem. We are not asking how difficult the problem $A\mathbf{x}=\mathbf{b}$ itself is to solve. That is, supposing we use the best solver available, how can we be sure our computed solution is accurate? That's the focus of this section.

Let's first consider an example where we take $A$ to be the **Hilbert matrix**. We'll denote this matrix by $H$. This is a special type of square matrix where the row $i$ column $j$ entry $h_{ij}$ is
$$ h_{ij} = \dfrac{1}{i+j-1}.$$  

Let's see what happens when we try to solve the system $H\mathbf{x}=\mathbf{b}$. Let $\mathbf{x}_n$ be the computed solution when $H$ is $n\times n$, and let $\hat{\mathbf{x}}_n$ be the exact solution. Now, it's not a trivial task to compute the exact solution $\hat{\mathbf{x}}_n$ for a random vector $\mathbf{b}$. A shortcut is to use a technique that some people call the method of *manufactured solutions*, where we pick an exact solution, $\mathbf{x}_n$, compute $H\mathbf{x}_n$, and set $\mathbf{b}$ equal to the result. This way we know what the exact solution is, and we can study how the error changes as the dimension of the system changes. Following our book's lead, we'll take $\mathbf{x}_n = [1 \hspace{0.5em} 1 \hspace{0.5em} \cdots \hspace{0.5em} 1] ^T$.

We'll use the LU method with partial pivoting. However, since $H$ is symmetric (this should be clear) and positive definite (less obvious), we could also use the Cholesky factorization.

In [58]:
#@title solving_hilbert_system.m
%%writefile solving_hilbert_system.m

% SET SIZE OF SYSTEM. WHAT HAPPENS AS N INCREASES?
N = 4; err_arr = [];

for n = 1:N
  % CONSTRUCT HILBERT MATRIX H
  v = 0:n-1 ;
  H = 1 ./ (v + [1:n]') ;
  % SET EXACT SOLUTION AND RIGHT HAND SIDE
  x = ones(n,1); b = H*x;

  % COMPUTE LU FACTORIZATION OF H WITH PARTIAL PIVOTING
  [L,U,P] = lu(H);

  % SOLVE TRIANGULAR SYSTEMS WITH BACKSLASH OPERATOR
  y = L \ (P*b);
  x_hat = U \ y;

  % COMPUTE RELATIVE ERROR AND ADD TO ARRAY
  err_arr = [err_arr norm(x - x_hat)/norm(x)];

  fprintf("Err for n = %g: %.3e \n",n,err_arr(end));
end


Overwriting solving_hilbert_system.m


In [ ]:
!octave -W solving_hilbert_system.m

What's going on here? For any dimension $n$, the Hilbert matrix of size $n$ is nonsingular (since it's positive definite), so we should be able to compute the solution without issue. The key is that $H$ is nonsingular in *exact arithmetic*. In finite precision, $H$ may be numerically singular, and this leads to inaccurate computed solutions $\mathbf{x}_n$. Thus, when solving $A\mathbf{x}=\mathbf{b}$ numerically, we are really solving the perturbed system

$$(A+\delta A)\mathbf{x} = \mathbf{b}+\delta\mathbf{b}.$$

The goal is to obtain an upper bound on $\|\mathbf{x} - \hat{\mathbf{x}}\|/\|\mathbf{x}\|$. Here, $\|\mathbf{x}\|$ denotes the **Euclidean norm** of the vector $\mathbf{x}$. This is a generalization of the Pythagorean theorem for vectors of length $n\geq 2$. Specifically, if $\mathbf{x} = [x_1\hspace{0.5em} x_2 \hspace{0.5em} \cdots \hspace{0.5em} x_n]^T$, then

$$ \|\mathbf{x}\| = \sqrt{x_1^2+x_2^2+\cdots+x_n^2}$$

To understand the essential ideas, while keeping the analysis as simple as possible, we'll assume we can store the matrix $A$ exactly, so that $\delta A = 0$. We will therefore analyze

$$A\mathbf{x} = \mathbf{b}+\delta\mathbf{b}.$$

We'll make the additional assumption that $A$ is symmetric and positive definite (SPD). In a sense, this means we're analyzing one the "nicest" systems possible. So whatever bound we get for $\|\mathbf{x} - \hat{\mathbf{x}}\|/\|\mathbf{x}\|$ for an SPD system is the best we can expect for a general system. Here we take $\mathbf{x}$ to be the exact solution to the unperturbed system $A\mathbf{x}=\mathbf{b}$, and $\hat{\mathbf{x}}$ is the solution of the perturbed system $A\mathbf{x} = \mathbf{b}+\delta\mathbf{b}.$ This gives us the two equations

$$
\begin{align}
A\mathbf{x}&=\mathbf{b} \\ A\hat{\mathbf{x}} &= \mathbf{b}+\delta\mathbf{b}
\end{align}
$$

Subtracting the second from the first gives

$$ A(\mathbf{x}-\hat{\mathbf{x}}) = -\delta\mathbf{b}. $$

Since we assumed $A$ is SPD, $A$ has an inverse $A^{-1}$. Applying $A^{-1}$ to both sides of $A(\mathbf{x}-\hat{\mathbf{x}})= -\delta\mathbf{b}$ and taking the norm of boths sides leave us with

$$\|\mathbf{x}-\hat{\mathbf{x}}\| = \|A^{-1}\delta\mathbf{b}\|.$$

To bound the error, it now suffices to bound $\|A^{-1}\delta\mathbf{b}\|$. This is where the assumption that $A$ is SPD makes the analysis a bit easier. A very, very, nice property of an $n\times n$ SPD matrices is that they have $n$ eigenvectors that form a basis for the space of vectors of length $n$. To be precise, since $A$ is an SPD $n\times n$ matrix, there are $n$ vectors, the **eigenvectors** of $A$, $\mathbf{v}_1$, $\mathbf{v}_2$, ... , $\mathbf{v}_n$ and $n$ scalars (greater than zero) $\lambda_1$, $\lambda_2$, ..., $\lambda_n$, the **eigenvalues** of $A$, such that

$$ A\mathbf{v}_i = \lambda \mathbf{v}_i,\hspace{1em} i=1,...,n$$

and for any vector $\mathbf{y}$, there are scalars $\alpha_1,...,\alpha_n$ such that $\mathbf{y} = \sum_{i=1}^n \alpha_i\mathbf{v}_i$. We'll need one more observation. Since $A\mathbf{v}_i = \lambda\mathbf{v}_i$, if we divide both sides by $\lambda_i$ and multiply both sides by $A^{-1}$ we find that

$$ A^{-1}\mathbf{v}_i = \lambda_i^{-1}\mathbf{v}_i$$

Now we're ready to find a bound on $\|A^{-1}\delta\mathbf{b}\|$. We'll go step-by-step.

1. Since the eigenvalues of $A$ form a basis for the space of vectors of length $n$, there are scalars $\alpha_1$,...,$\alpha_n$ such that

$$ \delta\mathbf{b} = \sum_{i=1}^n \alpha_i\mathbf{v}_i.$$

2. Multiply by $A^{-1}$.

$$ A^{-1}\delta\mathbf{b} = \sum_{i=1}^n \alpha_i A^{-1}\mathbf{v}_i = \sum_{i=1}^n \dfrac{\alpha_i}{\lambda_i} \mathbf{v}_i.$$

3. Take the norm and square it.  

$$ \|A^{-1}\delta\mathbf{b}\|^2 = \|\sum_{i=1}^n \dfrac{\alpha_i}{\lambda_i} \mathbf{v}_i\|^2 = \sum_{i=1}^n \dfrac{\alpha_i^2}{\lambda_i^2}.$$

The last equality follows since the eigenvectors of $A$ are *mutually orthogonal*, which means $\mathbf{v}_i \cdot \mathbf{v}_j = \delta_{ij}$, and $\|\mathbf{v}_i\| = 1$ for each $i=1,...,n$. This also implies that

$$\|\delta\mathbf{b}\|^2 = \delta\mathbf{b}\cdot \delta\mathbf{b}=\sum_{i=1}^n \alpha_i^2$$

4. Now, for each $i=1,...n$, $1/\lambda_i^2 \leq 1/\lambda_{min}$^2, where $\lambda_{min}$ is the eigenvalue of $A$ of smallest magnitude. It follows that

$$\|A^{-1}\delta\mathbf{b}\|^2 \leq \dfrac{1}{\lambda_{min}^2}\sum_{i=1}^n \alpha_i^2 = \dfrac{\|\delta\mathbf{b}\|^2}{\lambda_{min}^2}.$$

Take square roots.

$$\|\mathbf{x}-\hat{\mathbf{x}}\| = \|A^{-1}\delta\mathbf{b}\| \leq \dfrac{\|\delta\mathbf{b}\|}{\lambda_{min}}$$

5. At this point we've bounded the absolute error between the exact solution to $A\mathbf{x}=\mathbf{b}$, and the exact solution $\hat{\mathbf{x}}$ to the perturbed problem $A\mathbf{x} = \mathbf{b}+\delta\mathbf{b}$. To bound the relative error $\|\mathbf{x}-\hat{\mathbf{x}}\|/\|\mathbf{x}\|$, we need a lower bound on $\|\mathbf{x}\|$. Since $A\mathbf{x} = \mathbf{b}$, and $A$ is nonsingular, $\mathbf{x} = A^{-1}\mathbf{b}$. Now we can go through the same steps we sent through when we bounded $\|A^{-1}\delta\mathbf{b}\|$. Since $\mathbf{b}$ is a vector of length $n$, there are scalars $\beta_1$, ..., $\beta_n$ such that

$$ \mathbf{b} = \sum_{i=1}^n \beta_i \mathbf{v}_i$$

Thus

$$ \|A^{-1}\mathbf{b}\|^2 = \sum_{i=1}^n \dfrac{\beta_i^2}{\lambda_i^2} \geq \dfrac{\|\mathbf{b}\|^2}{\lambda_{max}^2}$$

here $\lambda_{max}$ is the eigenvalue of $A$ with largest magnitude. It follows that

$$ \dfrac{\|\mathbf{x} -\hat{\mathbf{x}}\|}{\|\mathbf{x}\|} = \dfrac{\|A^{-1}\delta\mathbf{b}\|}{\|A^{-1}\mathbf{b}\|} \leq \dfrac{\lambda_{max}}{\lambda_{min}}\dfrac{\|\delta\mathbf{b}\|}{\|\mathbf{b}\|}$$

We did it! We've got an upperbound on the relative error. How do we interpret it? Recall back in the first unit, we had the notions of *backward error*, *forward error*, and the *condition number* of a problem. Here, we want to compute $f(\mathbf{b}) = A^{-1}\mathbf{b}$. The exact solution is $f(\mathbf{b}) = \mathbf{x}$. We've actually somputed $\hat{\mathbf{x}} = f(\mathbf{b}+\delta\mathbf{b})$. So the relative *forward error* is the left hand side $\|\mathbf{x}-\hat{\mathbf{x}}\|/\|\mathbf{x}\|$. The relative backward error is the difference between the inputs $\mathbf{b}$ that yields the exact answer and the input $\mathbf{b}+\delta\mathbf{b}$ that gives the computed answer: $\|\delta\mathbf{b}\|/\|\mathbf{b}\|$. This explains two of the three terms in the inequality

$$ \dfrac{\|\mathbf{x} -\hat{\mathbf{x}}\|}{\|\mathbf{x}\|} \leq \dfrac{\lambda_{max}}{\lambda_{min}}\dfrac{\|\delta\mathbf{b}\|}{\|\mathbf{b}\|}$$

but how do we interpret $\lambda_{max}/\lambda_{min}$? Recall from the first couple weeks that we had the approximate bound


forward error $\leq$ condition number $\times$ backward error

The condition number should be thought of as a magnifying factor. A large condition number means even if we have very small backward error, the forward error can be much larger. This is precisely what $\lambda_{max}/\lambda_{\min}$ does in

$$ \dfrac{\|\mathbf{x} -\hat{\mathbf{x}}\|}{\|\mathbf{x}\|} \leq \dfrac{\lambda_{max}}{\lambda_{min}}\dfrac{\|\delta\mathbf{b}\|}{\|\mathbf{b}\|}$$

so $\lambda_{max}/\lambda_{\min}$ is the condition number of the problem. In fact, we define the **condition number** of the matrix $A$ to be

$$ K(A) := \dfrac{\lambda_{max}}{\lambda_{min}}$$

Notice that $K(A)$ is large when $\lambda_{min}$ is close to 0, which is equivalent to $A$ being nearly singular. This explains what we saw earlier with the Hilbert matrix. As the dimension increases, the matrix becomes very nearly singular. Consequently, $\lambda_{min}\approx 0$, and thus the condition number is very large, resulting in the large forward error we observed.

This is also why you'll see the "matrix singular to machine precision" error when you run the code above for $ n > 10$.


In [57]:
#@title hilbert_condition_number.m
%%writefile hilbert_condition_number.m

% START WITH N=5, AND SEE WHAT HAPPENS AS YOU INCREASE N
N = 5; err_arr = [];

for n = 1:N
  % CONSTRUCT HILBERT MATRIX H
  v = 0:n-1 ;
  H = 1 ./ (v + [1:n]');

  % COMPUTE CONDITION NUMBER
  K = cond(H);
  fprintf("Condition number of %g x %g Hilbert matrix: %.3e\n",n,n,K)
end

Overwriting hilbert_condition_number.m


In [56]:
!octave -W hilbert_condition_number.m

Condition number of 1 x 1 Hilbert matrix: 1.000e+00
Condition number of 2 x 2 Hilbert matrix: 1.928e+01
Condition number of 3 x 3 Hilbert matrix: 5.241e+02
Condition number of 4 x 4 Hilbert matrix: 1.551e+04
Condition number of 5 x 5 Hilbert matrix: 4.766e+05


# Condition number of a matrix

There are various ways to compute the condition number of a generic matrix $A$. The MATLAB/Octave command **cond** computes the 2-norm condition number, which is defined as

$$ K_2(A) := \|A\|_2\,\|A^{-1}\|_2$$

where

$$ \|A\|_2 := \sqrt{\lambda_{max}(A^TA)}$$

where $\lambda_{max}(A^TA)$ is the largest eigenvalue of $A^TA$. $\|A\|_2$ is a *matrix norm*, which we haven't discussed formally yet. We'll revisit this briefly when we discussed the **singular value decomposition** for solving overdetermined systems.

If $K(A)$ is small, in some relative sense, then we say that $A$ is *well conditioned*. If $K(A)$ is very large, then $A$ is an *ill conditioned* matrix.

In a certain sense, for a well conditioned matrix $A$, if the *residual* $\|\mathbf{r}\|:=\|A\hat{\mathbf{x}} - \mathbf{b}\|$ is small, then the error $\|\mathbf{x}-\hat{\mathbf{x}}\|/\|\mathbf{x}\|$ is small. If $A$ is ill-conditioned, then a small residual does not necessarily imply small error. Indeed, since $\delta\mathbf{b} = A\hat{\mathbf{x}}-\mathbf{b}=\mathbf{r}$, it follows that

$$ \dfrac{\|\mathbf{x} -\hat{\mathbf{x}}\|}{\|\mathbf{x}\|} \leq \dfrac{\lambda_{max}}{\lambda_{min}}\dfrac{\|\delta\mathbf{b}\|}{\|\mathbf{b}\|} = K(A)\dfrac{\|\mathbf{r}\|}{\|\mathbf{b}\|} $$

So, when $A$ is ill conditioned, the condition number $K(A)$ is large, and it follows that even if $\|\mathbf{r}\|$ is small, $K(A)$ could magnify it so that the error is large.
